In [ ]:
#Copy-and-paste the code below to use as "set-up" when your optimization model uses Pyomo. 
#Uncomment the appropriate solver that you need.
#for reference, see https://colab.research.google.com/drive/1yGk8RB5NXrcx9f1Tb-oCiWzbxh61hZLI?usp=sharing

#installing and importing pyomo
!pip install -q pyomo
from pyomo.environ import *

###installing and importing specific solvers (uncomment the one(s) you need)
###glpk
!apt-get install -y -qq glpk-utils
###cbc
#!apt-get install -y -qq coinor-cbc
###ipopt
#!wget -N -q "https://ampl.com/dl/open/ipopt/ipopt-linux64.zip"
#!unzip -o -q ipopt-linux64
###bonmin
#!wget -N -q "https://ampl.com/dl/open/bonmin/bonmin-linux64.zip"
#!unzip -o -q bonmin-linux64
###couenne
#!wget -N -q "https://ampl.com/dl/open/couenne/couenne-linux64.zip"
#!unzip -o -q couenne-linux64
###geocode
#!wget -N -q "https://ampl.com/dl/open/gecode/gecode-linux64.zip"
#!unzip -o -q gecode-linux64

#Using the solvers:
#SolverFactory('glpk', executable='/usr/bin/glpsol')
#SolverFactory('cbc', executable='/usr/bin/cbc')
#SolverFactory('ipopt', executable='/content/ipopt')
#SolverFactory('bonmin', executable='/content/bonmin')
#SolverFactory('couenne', executable='/content/couenne')
#SolverFactory('gecode', executable='/content/gecode')

In [ ]:
import pandas as pd

In [ ]:
#reading orders data
order_unit_cost = pd.read_excel('Prescriptive Final Model.xlsx', sheet_name='Orders_Plant_Unit_Cost') 
order_unit_cost.head()

,Orders,PLANT01,PLANT02,PLANT03,PLANT04,PLANT05,PLANT06,PLANT07,PLANT08,PLANT09,PLANT10
0,1000001,561,646,640,655,659,512,582,530,698,561
1,1000002,626,645,595,526,589,559,556,541,548,595
2,1000003,512,531,563,595,523,682,603,666,542,568
3,1000004,622,504,659,594,529,553,506,646,564,601
4,1000005,530,654,559,519,676,625,693,683,505,627


In [ ]:
#reading units data
order_units = pd.read_excel('Prescriptive Final Model.xlsx', sheet_name='Order_Product_Quantity') 
order_units.head()

,Orders,ProductID,Units
0,1000001,8,40
1,1000002,5,70
2,1000003,3,50
3,1000004,8,50
4,1000005,8,40


In [ ]:
#reading warehouse operating costs data
warehouse_data = pd.read_excel('Prescriptive Final Model.xlsx', sheet_name='Warehouse_Data') 
warehouse_data.head()

,WID,operating_cost,min_qty,max_qty
0,PLANT01,5382,1000,12000
1,PLANT02,5237,1000,9000
2,PLANT03,5618,1000,10500
3,PLANT04,5225,1000,8500
4,PLANT05,5537,1000,9000


In [ ]:
#reading shipping cost per unit
order_freight_costs = pd.read_excel('Prescriptive Final Model.xlsx', sheet_name='Order_Plant_Unit_Freight_Rate') 
order_freight_costs.head()

,Orders,PLANT01,PLANT02,PLANT03,PLANT04,PLANT05,PLANT06,PLANT07,PLANT08,PLANT09,PLANT10
0,1000001,69,99,52,131,85,132,70,119,59,128
1,1000002,104,68,113,72,131,60,78,122,111,90
2,1000003,52,86,143,101,87,88,106,51,87,150
3,1000004,98,59,58,100,102,138,112,129,109,120
4,1000005,60,135,95,131,93,101,102,143,55,145


In [ ]:
#list of orders
orders = order_unit_cost.loc[:,'Orders'].tolist()
orders

[1000001,
 1000002,
 1000003,
 1000004,
 1000005,
 1000006,
 1000007,
 1000008,
 1000009,
 1000010,
 1000011,
 1000012,
 1000013,
 1000014,
 1000015,
 1000016,
 1000017,
 1000018,
 1000019,
 1000020,
 1000021,
 1000022,
 1000023,
 1000024,
 1000025,
 1000026,
 1000027,
 1000028,
 1000029,
 1000030,
 1000031,
 1000032,
 1000033,
 1000034,
 1000035,
 1000036,
 1000037,
 1000038,
 1000039,
 1000040,
 1000041,
 1000042,
 1000043,
 1000044,
 1000045,
 1000046,
 1000047,
 1000048,
 1000049,
 1000050,
 1000051,
 1000052,
 1000053,
 1000054,
 1000055,
 1000056,
 1000057,
 1000058,
 1000059,
 1000060,
 1000061,
 1000062,
 1000063,
 1000064,
 1000065,
 1000066,
 1000067,
 1000068,
 1000069,
 1000070,
 1000071,
 1000072,
 1000073,
 1000074,
 1000075,
 1000076,
 1000077,
 1000078,
 1000079,
 1000080,
 1000081,
 1000082,
 1000083,
 1000084,
 1000085,
 1000086,
 1000087,
 1000088,
 1000089,
 1000090,
 1000091,
 1000092,
 1000093,
 1000094,
 1000095,
 1000096,
 1000097,
 1000098,
 1000099,
 1000100,


In [ ]:
#list of warehouses
warehouses = warehouse_data.loc[:,'WID'].tolist()
warehouses

['PLANT01',
 'PLANT02',
 'PLANT03',
 'PLANT04',
 'PLANT05',
 'PLANT06',
 'PLANT07',
 'PLANT08',
 'PLANT09',
 'PLANT10']

In [ ]:
#converting manufacturing cost per unit to a list of lists
unit_costs = order_unit_cost.loc[:, 'PLANT01':'PLANT10'].values.tolist()
unit_costs

[[561, 646, 640, 655, 659, 512, 582, 530, 698, 561],
 [626, 645, 595, 526, 589, 559, 556, 541, 548, 595],
 [512, 531, 563, 595, 523, 682, 603, 666, 542, 568],
 [622, 504, 659, 594, 529, 553, 506, 646, 564, 601],
 [530, 654, 559, 519, 676, 625, 693, 683, 505, 627],
 [511, 604, 582, 695, 588, 651, 586, 643, 661, 656],
 [591, 601, 674, 550, 633, 651, 525, 538, 521, 686],
 [552, 683, 585, 564, 677, 627, 549, 574, 660, 501],
 [662, 632, 685, 539, 553, 629, 517, 564, 546, 564],
 [561, 674, 652, 512, 599, 510, 673, 615, 638, 626],
 [582, 551, 638, 510, 614, 609, 610, 534, 657, 518],
 [691, 513, 510, 590, 548, 662, 590, 541, 538, 500],
 [590, 593, 504, 521, 622, 633, 582, 554, 571, 554],
 [676, 568, 685, 583, 576, 662, 607, 602, 608, 697],
 [610, 584, 616, 639, 621, 528, 532, 684, 579, 699],
 [566, 523, 620, 574, 536, 595, 550, 501, 638, 512],
 [617, 615, 519, 599, 677, 533, 686, 662, 640, 560],
 [566, 560, 601, 532, 671, 630, 506, 554, 548, 595],
 [551, 653, 611, 665, 648, 608, 539, 632, 653,

In [ ]:
#converting fixed cost data to a list
operating_costs = warehouse_data.loc[:, 'operating_cost'].values.tolist()
operating_costs

[5382, 5237, 5618, 5225, 5537, 5005, 5230, 5179, 5820, 5850]

In [ ]:
#converting unit frieght cost to a list of lists
freight_costs = order_freight_costs.loc[:, 'PLANT01':'PLANT10'].values.tolist()
freight_costs

[[69, 99, 52, 131, 85, 132, 70, 119, 59, 128],
 [104, 68, 113, 72, 131, 60, 78, 122, 111, 90],
 [52, 86, 143, 101, 87, 88, 106, 51, 87, 150],
 [98, 59, 58, 100, 102, 138, 112, 129, 109, 120],
 [60, 135, 95, 131, 93, 101, 102, 143, 55, 145],
 [115, 112, 126, 120, 148, 63, 112, 126, 88, 76],
 [98, 59, 126, 105, 138, 112, 70, 97, 53, 134],
 [111, 87, 88, 150, 96, 131, 96, 53, 61, 76],
 [147, 77, 61, 125, 95, 124, 54, 83, 105, 51],
 [139, 58, 103, 76, 123, 146, 68, 86, 54, 60],
 [121, 76, 88, 136, 136, 132, 121, 76, 107, 148],
 [110, 78, 58, 97, 140, 107, 58, 58, 62, 53],
 [98, 63, 97, 85, 92, 134, 66, 133, 133, 99],
 [94, 126, 134, 59, 127, 137, 108, 109, 79, 94],
 [89, 73, 101, 116, 132, 108, 116, 52, 50, 64],
 [68, 74, 105, 99, 64, 59, 150, 67, 133, 139],
 [89, 82, 150, 85, 129, 108, 109, 129, 58, 146],
 [96, 72, 105, 52, 139, 116, 92, 77, 96, 110],
 [60, 114, 108, 104, 107, 78, 75, 92, 83, 143],
 [125, 126, 64, 135, 99, 131, 107, 139, 109, 85],
 [103, 132, 63, 125, 130, 58, 61, 78, 83,

In [ ]:
#converting units ordered data to a list
unit_demands = order_units.loc[:, 'Units'].values.tolist()
unit_demands

[40,
 70,
 50,
 50,
 40,
 80,
 50,
 50,
 50,
 60,
 100,
 90,
 60,
 90,
 40,
 100,
 40,
 80,
 70,
 60,
 60,
 50,
 50,
 50,
 100,
 80,
 80,
 40,
 80,
 100,
 60,
 90,
 90,
 60,
 70,
 60,
 90,
 100,
 80,
 90,
 100,
 40,
 70,
 90,
 70,
 40,
 70,
 50,
 100,
 100,
 60,
 60,
 70,
 70,
 60,
 60,
 80,
 80,
 70,
 90,
 60,
 50,
 40,
 40,
 90,
 40,
 60,
 40,
 70,
 70,
 90,
 40,
 70,
 60,
 100,
 80,
 60,
 40,
 70,
 70,
 60,
 60,
 60,
 100,
 90,
 90,
 80,
 70,
 100,
 80,
 80,
 50,
 60,
 90,
 70,
 40,
 40,
 100,
 40,
 90,
 60,
 50,
 40,
 90,
 50,
 80,
 60,
 40,
 80,
 90,
 60,
 40,
 50,
 100,
 90,
 40,
 50,
 80,
 80,
 60,
 50,
 60,
 60,
 40,
 80,
 50,
 60,
 70,
 90,
 40,
 70,
 70,
 80,
 80,
 70,
 100,
 100,
 50,
 80,
 40,
 40,
 40,
 60,
 100,
 70,
 80,
 100,
 90,
 80,
 90,
 80,
 90,
 70,
 70,
 100,
 100,
 100,
 70,
 90,
 60,
 100,
 90,
 40,
 40,
 50,
 80,
 40,
 80,
 80,
 70,
 40,
 40,
 90,
 60,
 70,
 60,
 100,
 40,
 90,
 100,
 90,
 60,
 90,
 90,
 90,
 90,
 90,
 60,
 50,
 50,
 60,
 80,
 40,
 50,
 70,
 4

In [ ]:
#converting min capacities data to a list
min_capacity = warehouse_data.loc[:, 'min_qty'].values.tolist()
min_capacity

[1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000]

In [ ]:
#converting max capacities data to a list
max_capacity = warehouse_data.loc[:, 'max_qty'].values.tolist()
max_capacity

[12000, 9000, 10500, 8500, 9000, 9500, 9500, 10000, 11000, 8500]

In [ ]:
#input number of orders and warehouses
num_orders = 1000 #indexed by i
num_warehouses = 10 #indexed by j

#define the optimization model
model = ConcreteModel()

#DVs
model.x = Var(range(num_warehouses), domain = Binary)
model.y = Var(range(num_orders), range(num_warehouses), domain = NonNegativeIntegers)

#Objective function
manufacturing_cost = sum(model.y[i,j]*unit_costs[i][j] for i in range(num_orders) for j in range(num_warehouses))
shipping_cost = sum(model.y[i,j]*freight_costs[i][j] for i in range(num_orders) for j in range(num_warehouses))
operating_cost = sum(model.x[i]*operating_costs[i] for i in range(num_warehouses))
total_cost = manufacturing_cost + shipping_cost + operating_cost
model.Objective = Objective(expr = total_cost, sense = minimize)

#Constraints

#to fullfill the demand, supply>= demand
model.supplydemand = ConstraintList()
for i in range(num_orders):
  model.supplydemand.add(sum(model.y[i,j] for j in range(num_warehouses)) >= unit_demands[i])

#demand per warehouse<= maximum capacity of that warehouse
model.maxsupplycapacity = ConstraintList()
for i in range(num_warehouses):
  model.maxsupplycapacity.add(sum(model.y[j,i] for j in range(num_orders)) <= model.x[i]*max_capacity[i])

#demand per warehouse>= minimum capacity of that warehouse
model.minsupplycapacity = ConstraintList()
for i in range(num_warehouses):
  model.minsupplycapacity.add(sum(model.y[j,i] for j in range(num_orders)) >= model.x[i]*min_capacity[i])

model.pprint()

In [ ]:
#solve the model
opt = SolverFactory('glpk')
opt.options['tmlim'] = 5 #specifies the time limit (in seconds)
opt.options['mipgap'] = .00 #specifies the optimality gap tolerance (.01 means can stop if <1% of optimal obj)
results = opt.solve(model, tee=True) #can set tee=True if you want to see the details.

GLPSOL: GLPK LP/MIP Solver, v4.65
Parameter(s) specified in the command line:
 --tmlim 5 --mipgap 0.0 --write /tmp/tmpej4ie7m_.glpk.raw --wglp /tmp/tmpqktojk8s.glpk.glp
 --cpxlp /tmp/tmpcm3fqdy4.pyomo.lp
Reading problem data from '/tmp/tmpcm3fqdy4.pyomo.lp'...
/tmp/tmpcm3fqdy4.pyomo.lp:63114: warning: lower bound of variable 'x1' redefined
/tmp/tmpcm3fqdy4.pyomo.lp:63114: warning: upper bound of variable 'x1' redefined
1021 rows, 10011 columns, 30021 non-zeros
10010 integer variables, 10 of which are binary
63124 lines were read
Writing problem data to '/tmp/tmpqktojk8s.glpk.glp'...
62088 lines were written
GLPK Integer Optimizer, v4.65
1021 rows, 10011 columns, 30021 non-zeros
10010 integer variables, 10 of which are binary
Preprocessing...
1020 rows, 10010 columns, 30020 non-zeros
10010 integer variables, 10 of which are binary
Scaling...
 A: min|aij| =  1.000e+00  max|aij| =  1.200e+04  ratio =  1.200e+04
GM: min|aij| =  5.373e-01  max|aij| =  1.861e+00  ratio =  3.464e+00
EQ: min|a

In [ ]:
#
actual_supply = [[model.y[j,i]() for i in range(num_warehouses)] for j in range(num_orders)]

#print the minimum costs
Final_cost = model.Objective()
print('Minimum Costs (Manufacturing+Operations+Shipping):', Final_cost)

Minimum Costs (Manufacturing+Operations+Shipping): 42253283.0


In [ ]:
#fetch the operating warehouses for optimal solutions
[model.x[i]() for i in range(num_warehouses)]

[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0]

In [ ]:
#converting the DV (y) that maps optimal supply quantity from order to warehouse
warehouse_supply = pd.DataFrame(actual_supply, index = orders, columns = warehouses)
warehouse_supply

,PLANT01,PLANT02,PLANT03,PLANT04,PLANT05,PLANT06,PLANT07,PLANT08,PLANT09,PLANT10
1000001,40.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1000002,0.0,0.0,0.0,70.0,0.0,0.0,0.0,0.0,0.0,0.0
1000003,50.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1000004,0.0,50.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1000005,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,40.0,0.0
...,...,...,...,...,...,...,...,...,...,...
1000996,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,40.0
1000997,0.0,0.0,0.0,0.0,0.0,0.0,90.0,0.0,0.0,0.0
1000998,0.0,60.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1000999,0.0,0.0,0.0,0.0,0.0,0.0,50.0,0.0,0.0,0.0


In [ ]:
#Fetch the capacity of warehouses fulfilled given the optimal solution
import numpy as np
warehouse_percent_capacity_fulfilled = pd.DataFrame(pd.DataFrame(warehouse_supply.sum(), columns = ['actual_supply'])['actual_supply']/warehouse_data[['max_qty']].set_index(np.array(warehouses))['max_qty']*100, columns = ['per_capacity_fulfilled'])
warehouse_percent_capacity_fulfilled.reset_index(inplace=True)
warehouse_percent_capacity_fulfilled.rename(columns = {'index':'WID'})

,WID,per_capacity_fulfilled
0,PLANT01,62.333333
1,PLANT02,89.888889
2,PLANT03,66.285714
3,PLANT04,74.705882
4,PLANT05,68.666667
5,PLANT06,66.000000
6,PLANT07,77.578947
7,PLANT08,66.500000
8,PLANT09,68.636364
9,PLANT10,83.647059


In [ ]:
#saving the final results as csv
warehouse_supply.to_csv('final_units_supply_solution.csv', index = False)

In [ ]:
warehouse_percent_capacity_fulfilled.to_csv('warehouse_percent_capcity_fulfilled_soution.csv', index = False)

In [ ]:
#downloading the files
from google.colab import files
files.download('final_units_supply_solution.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
from google.colab import files
files.download('warehouse_percent_capcity_fulfilled_soution.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>